<a href="https://colab.research.google.com/github/Kamilla1533/Fake-Vintage-Filter/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
from torchvision.utils import save_image
from PIL import Image
import numpy as np
import os
import lpips
import matplotlib.pyplot as plt

In [ ]:
MODELS_DIR = '/content/drive/MyDrive/stylegan_models'

In [5]:
CHECKPOINT_NUMBER = 100000

In [ ]:
INPUT_IMAGE_PATH = '/content/drive/MyDrive/my_test_photo.jpg'

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/stylegan_results/vintage_projections'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
LEARNING_RATE = 0.01
NUM_ITERATIONS = 1000
LAMBDA_LPIPS = 0.8
LAMBDA_L2 = 0.2
TRUNC_PSI = 0.7

# Загрузка обученной модели

In [ ]:
%cd /content/

In [ ]:
!git clone https://github.com/lucidrains/stylegan2-pytorch.git

In [ ]:
%cd stylegan2-pytorch

In [ ]:
from stylegan2_pytorch import ModelLoader

In [ ]:
loader = ModelLoader(
    base_dir=MODELS_DIR,
    name='my_vintage_filter_project',
    load_from=CHECKPOINT_NUMBER,
    image_size=128
)

In [ ]:
generator = loader.G.cuda()
generator.eval()

# Загрузка фото

In [ ]:
try:
    input_image_pil = Image.open(INPUT_IMAGE_PATH).convert("RGB")
    input_image_pil = input_image_pil.resize((loader.image_size, loader.image_size), Image.LANCZOS)
    input_image_np = np.array(input_image_pil).transpose(2, 0, 1)
    input_image_tensor = torch.from_numpy(input_image_np).float() / 255.0
    input_image_tensor = (input_image_tensor * 2) - 1
    input_image_tensor = input_image_tensor.unsqueeze(0).cuda()

    print(f"Входное изображение '{INPUT_IMAGE_PATH}' успешно загружено и подготовлено.")

except Exception as e:
    print(f"Ошибка при загрузке или подготовке входного изображения: {e}")
    exit()

# Инициализация случайного латентного вектора

In [ ]:
latent_vector_z = torch.randn(1, loader.latent_dim, requires_grad=True, device='cuda')

In [ ]:
optimizer = torch.optim.Adam([latent_vector_z], lr=LEARNING_RATE)

In [ ]:
for i in range(NUM_ITERATIONS):
    optimizer.zero_grad()
    styles = loader.noise_to_styles(latent_vector_z, trunc_psi=TRUNC_PSI)
    generated_image = loader.styles_to_images(styles)
    loss_lpips = loss_fn_lpips(generated_image, input_image_tensor).mean()
    loss_l2 = torch.mean((generated_image - input_image_tensor)**2)
    total_loss = LAMBDA_LPIPS * loss_lpips + LAMBDA_L2 * loss_l2

    total_loss.backward()
    optimizer.step()

In [ ]:
if (i + 1) % 100 == 0 or i == 0:
        print(f"Итерация {i+1}/{NUM_ITERATIONS}, Общая потеря: {total_loss.item():.4f}, LPIPS: {loss_lpips.item():.4f}, L2: {loss_l2.item():.4f}")

# Сохранение фото

In [ ]:
final_generated_image = generated_image.detach().cpu()

In [ ]:
final_image_to_save = (final_generated_image + 1) * 0.5

In [ ]:
output_image_name = os.path.splitext(os.path.basename(INPUT_IMAGE_PATH))[0] + "_vintage_projected.jpg"
output_full_path = os.path.join(OUTPUT_DIR, output_image_name)
save_image(final_image_to_save, output_full_path)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(input_image_pil)
axes[0].set_title("Исходное фото")
axes[0].axis('off')

In [ ]:
display_generated_image = (final_generated_image.squeeze(0).permute(1, 2, 0).numpy() * 255).astype(np.uint8)
axes[1].imshow(display_generated_image)
axes[1].set_title("Винтажное фото (проекция StyleGAN)")
axes[1].axis('off')

plt.show()